# Ejercicios Pair Programming - Modulo 2 - Sprint 2 
## Lección 1 - Limpieza II: Strings
### Ana Gonzalez y Sonia Ruiz

De todo esto, nos damos cuenta que solo tenemos que limpiar algunas de las columnas, en concreto age, species, country, fatal, year, sex.       
Si reducimos esto a una tabla para saber que tenemos ya limpito y que no.       
**Actualizamos esta tabla ya que en el ejercicio de pair de Limpieza I ya dejamos algunas columnas limpitas:

|variable|¿Está limpia?|
|---|---|
|age|❌ esta en formato string cuando debería ser integer y en algunos casos tenemos rangos de edad   |
|species   |❌ es un jaleo! Debemos unificar los nombres y reducir a las especies más importantes   |
|country   |✔️ los paises están en mayúsculas, algunos se repiten con algunos cambios   |
|fatal   |✔️ la limpiamos en el pair de Pandas V    |
|year   |✔️ es una columna de tipo float deberíamos convertirla a integer   |
|sex   |✔️ la limpiamos en el pair de Pandas V  |
|fecha   |✔️ la limpiamos en el pair de Pandas V   |

Por lo tanto, en este ejercicio nos queda por limpiar las columnas de age y species, así que manos a la obra. Vamos a ver que podemos hacer para cada una de las columnas y os dejaremos algunas pistas 😉. Para esto trabajaremos con el fichero generado en Limpieza I.

1. Columna de species: si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna. Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (White), el tiburón tigre (Tiger ), el tiburón gris (Grey), el tiburón limón (Lemon) y el tiburón toro (Bull). El resto de las especies las incluiremos en un único grupo que podremos llamar "Unspecified". Ahora nos podemos sentir un poco abrumadas y no saber como enfrentarnos a este reto, pero don't worry, os dejamos por aquí unas 💡 **pistas** 💡 para que os ayuden a entender cómo hacerlo.

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df=pd.read_csv('files/attacks_limp1.csv', index_col=0)
df.head(2)

,year,country,age,species_,month,dead,gender
0,2018,usa,57,White shark,Jun,N,F
1,2018,brazil,18,Tiger shark,Jun,Y,M


- Los valores de las columnas son strings por lo que podremos usar regex para buscar palabras clave en cada celda y asignarlo a una de las categorías que hemos definido previamente.

In [48]:
# imaginemos que el valor de una celda es el siguiente

'White shark, 3.5 m'

# tendremos que buscar el patrón de regex que nos permita extraer White shark de ese string y que nos devuelva solo White shark. 
# Un patrón que podriamos usar es: 

patron_blanco = r".*[Ww](hite|HITE).*" # esto podría ser así porque puede estar en mayúsculas o en minúsculas. 

# de la misma forma que hemos sacado el patron para el tiburón blanco, tendremos que sacar los patrones para las otras 4 especies
#  que queremos "encontrar". 

- Tendremos que crearnos una función que aplicaremos sobre nuestra columna species_ para que nos devuelva una nueva columna con los valores clasificados en función de los patrones de regex que hayamos definido.

In [3]:
def obtener_especie( col):
    patron1=r".*[Ww](hite|HITE).*"
    patron2=r".*[Tt](iger|IGER).*"
    patron3=r".*[Gg](rey|REY).*"
    patron4=r".*[Ll](emon|EMON).*"
    patron5=r".*[Bb](ull|ULL).*"
    try:
        if len(re.findall(patron1,col))>0:
            return 'white'
        elif len(re.findall(patron2,col))>0:
            return 'tiger'
        elif len(re.findall(patron3,col))>0:
            return 'grey'
        elif len(re.findall(patron4,col))>0:
            return 'lemon'
        elif len(re.findall(patron5,col))>0:
            return 'bull'
        else:
            return 'unspecified'
    except:
        return np.nan

In [4]:
df['specie']=df['species_'].apply(obtener_especie)
df.head()

,year,country,age,species_,month,dead,gender,specie
0,2018,usa,57,White shark,Jun,N,F,white
1,2018,brazil,18,Tiger shark,Jun,Y,M,tiger
2,2018,usa,15,"Bull shark, 6'",May,N,M,bull
3,2018,australia,32,Grey reef shark,May,N,M,grey
4,2018,england,21,Invalid incident,May,N,M,unspecified


2. Columna de age: es una columna de tipo string pero debería ser de tipo integer. Además, en esta columna nos vamos a encontrar con algunos errores tipográficos, estos incluyen:       
- Edad en formato string      
- Edades separadas por &, or, to, >       
- Edades con ?        

Vamos con algunas 💡 **pistas** 💡 para que os ayuden a entender cómo hacerlo:
- Primero tendremos que eliminar todos esos símbolos especiales que nos aparecen. De nuevo, podremos usar regex para extraer unicamente los números que es lo que nos interesa. Usar este regex en una función para sacar solo los números.

- Puede que os salga un error similar a este:
    ``` python
    TypeError: expected string or bytes-like object
    ```
    Para solucionar este problema, antes de nada tendréis que ejecutar este código para que no os de error:
    ``` python
    df['nombre_columna'] = df['nombre_columna'].astype(str)
    ```     


- Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos. Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. El método explode, [aqui](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.explode.html).

In [5]:
#Creamos la funcion para limpiar la columna age.
def años (col):
    try:
        int(col)
        return int(col)       
    except:
        try:
            list= re.findall('\d+',col)
            if len(list)>1:
                suma=0
                for i in list:
                    suma+=int(i)
                    longitud=len(list)
                    return suma/longitud
            elif len(list)==1:
                return int(list[0])
            else:
                return np.nan
        except:
            return np.nan

In [6]:
#Aplicamos la funcion para limpiar la columna age y creamos la columna edad con su resultado para cada celda.
df['edad']=df['age'].apply(años)
df.head()

,year,country,age,species_,month,dead,gender,specie,edad
0,2018,usa,57,White shark,Jun,N,F,white,57.0
1,2018,brazil,18,Tiger shark,Jun,Y,M,tiger,18.0
2,2018,usa,15,"Bull shark, 6'",May,N,M,bull,15.0
3,2018,australia,32,Grey reef shark,May,N,M,grey,32.0
4,2018,england,21,Invalid incident,May,N,M,unspecified,21.0


In [15]:
#Comprobamos el numero de registros no nulos que tenemos antes de limpiar la columna.
df['age'].value_counts().sum()

1516

In [8]:
#Comprobamos el numero de valores nulos que tenemos en la columna antes de limpiarla.
df['age'].isnull().sum()

130

In [9]:
#Comprobamos el numero de registros no nulos que tenemos despues de limpiar la columna.
df['edad'].value_counts().sum()

1512

In [10]:
#Comprobamos el numero de valores nulos que tenemos en la columna despues de limpiarla.
df['edad'].isnull().sum()

134

In [57]:
#Tenemos 4 valores nulos más en al columna nueva unificada de edades. Esto se debe a que algunos
#  de los valores limpiados se han registrado como nulos ya que no contenian ningun numero (por ejemplo, 'young').

In [17]:
df['age'].unique()

array(['57', '18', '15', '32', '21', '30', '60', '33', '19', '25', '10',
       '69', '55', '35', '20', '54', '22', '31', '40', '37', '11', '13',
       '34', '50', '46', '48', '17', '28', '65', '73', '58', '36', '60s',
       '51', '61', '59', '42', '6', '27', '23', '29', '39', '24', '12',
       '26', '71', '43', '9', '44', '14', '62', '52', '38', '68', '16',
       '47', '63', '70', '41', '40s', '53', '20s', '7', '66', '45', '74',
       '64', '8', '56', '49', '18 or 20', 'Teen', '30s', '77', '8 or 10',
       '84', '\xa0 ', ' ', '30 or 36', '6½', '5', ' 30', ' 28', "60's",
       '67', '>50', '? & 19', '21, 34,24 & 35', '30 & 32', '13 or 18',
       '7 or 8', '9 or 10', nan, 'young', '13 or 14'], dtype=object)

In [18]:
df['edad'].unique()

array([57.  , 18.  , 15.  , 32.  , 21.  , 30.  , 60.  , 33.  , 19.  ,
       25.  , 10.  , 69.  , 55.  , 35.  , 20.  , 54.  , 22.  , 31.  ,
       40.  , 37.  , 11.  , 13.  , 34.  , 50.  , 46.  , 48.  , 17.  ,
       28.  , 65.  , 73.  , 58.  , 36.  , 51.  , 61.  , 59.  , 42.  ,
        6.  , 27.  , 23.  , 29.  , 39.  , 24.  , 12.  , 26.  , 71.  ,
       43.  ,  9.  , 44.  , 14.  , 62.  , 52.  , 38.  , 68.  , 16.  ,
       47.  , 63.  , 70.  , 41.  , 53.  ,  7.  , 66.  , 45.  , 74.  ,
       64.  ,  8.  , 56.  , 49.  ,   nan, 77.  ,  4.  , 84.  ,  5.  ,
       67.  ,  5.25,  6.5 ,  3.5 ,  4.5 ])

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1646 entries, 0 to 1671
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   year      1646 non-null   int64  
 1   country   1636 non-null   object 
 2   age       1516 non-null   object 
 3   species_  1543 non-null   object 
 4   month     1491 non-null   object 
 5   dead      1547 non-null   object 
 6   gender    1631 non-null   object 
 7   specie    1543 non-null   object 
 8   edad      1512 non-null   float64
dtypes: float64(1), int64(1), object(7)
memory usage: 128.6+ KB


- Por último cambiad el tipo de la columna de string a integer.

In [59]:
df['edad']=df['edad'].astype('int64')

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

`Nuestra columna 'edad' está tipo float, debido a la funcion que le hemos aplicado, ya que le hemos dejado valores nulos, por tanto toda la columna es tipo float y no es posible pasarla a integer.`

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1646 entries, 0 to 1671
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   year      1646 non-null   int64  
 1   country   1636 non-null   object 
 2   age       1516 non-null   object 
 3   species_  1543 non-null   object 
 4   month     1491 non-null   object 
 5   dead      1547 non-null   object 
 6   gender    1631 non-null   object 
 7   specie    1543 non-null   object 
 8   edad      1512 non-null   float64
dtypes: float64(1), int64(1), object(7)
memory usage: 128.6+ KB


In [61]:
df.head()

,year,country,age,species_,month,dead,gender,specie,edad
0,2018,usa,57,White shark,Jun,N,F,white,57.0
1,2018,brazil,18,Tiger shark,Jun,Y,M,tiger,18.0
2,2018,usa,15,"Bull shark, 6'",May,N,M,bull,15.0
3,2018,australia,32,Grey reef shark,May,N,M,grey,32.0
4,2018,england,21,Invalid incident,May,N,M,unspecified,21.0


In [62]:
df['edad'].unique()

array([57.  , 18.  , 15.  , 32.  , 21.  , 30.  , 60.  , 33.  , 19.  ,
       25.  , 10.  , 69.  , 55.  , 35.  , 20.  , 54.  , 22.  , 31.  ,
       40.  , 37.  , 11.  , 13.  , 34.  , 50.  , 46.  , 48.  , 17.  ,
       28.  , 65.  , 73.  , 58.  , 36.  , 51.  , 61.  , 59.  , 42.  ,
        6.  , 27.  , 23.  , 29.  , 39.  , 24.  , 12.  , 26.  , 71.  ,
       43.  ,  9.  , 44.  , 14.  , 62.  , 52.  , 38.  , 68.  , 16.  ,
       47.  , 63.  , 70.  , 41.  , 53.  ,  7.  , 66.  , 45.  , 74.  ,
       64.  ,  8.  , 56.  , 49.  ,   nan, 77.  ,  4.  , 84.  ,  5.  ,
       67.  ,  5.25,  6.5 ,  3.5 ,  4.5 ])

3. Guarda el csv con las columnas limpias para seguir trabajando con este dataframe limpio.

In [65]:
df.to_csv('files/attacks_limp2.csv')

**Happy coding**🦈